In [1]:
# Import Dependencies
import os
import pandas as pd


In [2]:
# Path to file directory and variables for the two files.
file_dir = os.path.join("Data")

# imdb Titles metadata
titles_metadata_file = f'{file_dir}/title_basics_non-adult_movies.tsv'

# imdb Ratings data
ratings_data_file = f'{file_dir}/title_ratings.csv'

In [7]:
# Import imdb Titles metadata, and imdb Ratings data

titles_metadata = pd.read_csv(titles_metadata_file, sep='\t')
ratings_data = pd.read_csv(ratings_data_file)


In [11]:
# Check titles_metadata DataFrame
print(titles_metadata.shape
titles_metadata.count()
titles_metadata.head()


TypeError: 'tuple' object is not callable

In [10]:
# Check titles_metadata DataFrame
ratings_data.count()
ratings_data.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1834
1,tt0000002,6.0,236
2,tt0000003,6.5,1594
3,tt0000004,6.0,153
4,tt0000005,6.2,2410
